In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
HOME = "/content/drive/My Drive/template_experiment/gpt3.5/prompt1/zero/" # @param {type:"string"}
ANNOTATOR = "Paul" # @param {type:"string"}

In [3]:
import pandas as pd
import numpy as np
import os
import statistics

In [4]:
df_dev_results = {}

for i in range(1, 11):
    file_name = f"dev_predictions_zero-shot_{i-1}.csv"
    file_path = os.path.join(HOME, file_name)
    df_dev_results[f"run {i}"] = pd.read_csv(file_path)

for k in df_dev_results:
    df_dev_results[k]["predicted_slot_fillers"] = df_dev_results[k]["predicted_slot_fillers"].apply(eval)

# Template selection

In [8]:
from sklearn.metrics import classification_report, precision_recall_fscore_support

result_rows = []

for ft in df_dev_results["run 1"]["Fallacy Types"].unique():
    def eval_F(df, col):
        _p, _r, _f, _ = precision_recall_fscore_support(
            df[df["Fallacy Types"] == ft]["Template #"],
            df[df["Fallacy Types"] == ft][col],
            average="macro",
            )
        return _f

    result_rows.append({
        "Fallacy Type": ft,
        "Majority":eval_F(df_dev_results["run 1"], "predicted_template_no_bl_majority"),
        "run 1": eval_F(df_dev_results["run 1"], "predicted_template_no"),
        "run 2": eval_F(df_dev_results["run 2"], "predicted_template_no"),
        "run 3": eval_F(df_dev_results["run 3"], "predicted_template_no"),
        "run 4": eval_F(df_dev_results["run 4"], "predicted_template_no"),
        "run 5": eval_F(df_dev_results["run 5"], "predicted_template_no"),
        "run 6": eval_F(df_dev_results["run 6"], "predicted_template_no"),
        "run 7": eval_F(df_dev_results["run 7"], "predicted_template_no"),
        "run 8": eval_F(df_dev_results["run 8"], "predicted_template_no"),
        "run 9": eval_F(df_dev_results["run 9"], "predicted_template_no"),
        "run 10": eval_F(df_dev_results["run 10"], "predicted_template_no"),
    })

result_rows.append({
    "Fallacy Type": "Overall",
    "Majority": np.mean(result_rows[-1]["Majority"]),
    "run 1": np.mean([row["run 1"] for row in result_rows]),
    "run 2": np.mean([row["run 2"] for row in result_rows]),
    "run 3": np.mean([row["run 3"] for row in result_rows]),
    "run 4": np.mean([row["run 4"] for row in result_rows]),
    "run 5": np.mean([row["run 5"] for row in result_rows]),
    "run 6": np.mean([row["run 6"] for row in result_rows]),
    "run 7": np.mean([row["run 7"] for row in result_rows]),
    "run 8": np.mean([row["run 8"] for row in result_rows]),
    "run 9": np.mean([row["run 9"] for row in result_rows]),
    "run 10": np.mean([row["run 10"] for row in result_rows]),
})

pd.options.display.float_format = "{:.2f}".format
pd.DataFrame(result_rows).transpose()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

,0,1,2,3,4
Fallacy Type,False Dilemma,Faulty Generalization,False Causality,Fallacy of Credibility,Overall
Majority,0.21,0.09,0.11,0.10,0.10
run 1,0.04,0.30,0.00,0.12,0.11
run 2,0.04,0.10,0.02,0.13,0.07
run 3,0.04,0.21,0.06,0.19,0.12
run 4,0.04,0.16,0.01,0.13,0.08
run 5,0.04,0.12,0.04,0.15,0.09
run 6,0.04,0.17,0.11,0.12,0.11
run 7,0.04,0.32,0.02,0.13,0.13
run 8,0.04,0.16,0.02,0.17,0.10


In [9]:
FD = [result_rows[0]['run {}'.format(i)] for i in range(1,11)]
false_dilemma = sum(FD) / len(FD)
fd_deviation = statistics.stdev(FD)

FG = [result_rows[1]['run {}'.format(i)] for i in range(1,11)]
generalization = sum(FG) / len(FG)
fg_deviation = statistics.stdev(FG)

FC = [result_rows[2]['run {}'.format(i)] for i in range(1,11)]
false_causality = sum(FC) / len(FC)
fc_deviation = statistics.stdev(FC)

FCred = [result_rows[3]['run {}'.format(i)] for i in range(1,11)]
cred = sum(FCred) / len(FCred)
fcred_deviation = statistics.stdev(FCred)

overall = [result_rows[4]['run {}'.format(i)] for i in range(1,11)]
average = sum(overall) / len(overall)
ovl_deviation = statistics.stdev(overall)

print(false_dilemma)
print(generalization)
print(false_causality)
print(cred)
print(average)

0.03642759232381874
0.18386745337257265
0.0428263049629231
0.14746834113823554
0.10264742294938752


In [10]:
print(fd_deviation)
print(fg_deviation)
print(fc_deviation)
print(fcred_deviation)
print(ovl_deviation)

0.002684137634699006
0.07342351804742481
0.036197461134071934
0.02766609812410728
0.01871682939178996


## Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

df = df_dev_results["5-shot"]

for ft in df_dev_results["zero-shot"]["Fallacy Types"].unique():
    print(ft)
    print(confusion_matrix(
        df[df["Fallacy Types"] == ft]["Template #"],
        df[df["Fallacy Types"] == ft]["predicted_template_no"],
        labels=range(1,6),
        ))
    print()

False Dilemma
[[ 3  0  2  0  1]
 [ 5  3  4  1 23]
 [ 0  0  0  0  0]
 [ 0  1  2  1  0]
 [ 0  0  0  0  4]]

Faulty Generalization
[[ 1  1  0  0  7]
 [ 0  6  1  0  7]
 [ 0  1  5  0  4]
 [ 0  0  0  0  2]
 [ 1  0  1  1 12]]

False Causality
[[ 1  1  0  1  1]
 [ 5  7  1  0  7]
 [ 1  2 11  1  3]
 [ 3  1  1  0  2]
 [ 1  0  0  0  0]]

Fallacy of Credibility
[[ 3  0  1  0  1]
 [ 1  1  3  0  2]
 [ 2  0 10  0  5]
 [ 1  2  0  1  0]
 [ 5  0  5  0  7]]



# Slot filling

In [11]:
def slotfiller_match(row, slotfillers, strict=False):
    def overlap(s1, s2):
        return len(set(s1.lower().split()) & set(s2.lower().split())) / len(set(s1.lower().split()) | set(s2.lower().split()))

    for k in "A A' C C' X".split():
        y_true = row["{} ({})".format(k, ANNOTATOR)]
        y_pred = slotfillers.get(k, "")

        if not pd.notna(y_true): continue

        if strict:
            if y_pred.lower() != y_true.lower():
                return False

        else:
            if overlap(y_pred, y_true) < 0.5:
                return False

    return True

def slot_filler_score(df, pred_slot_fillers, strict=False):
    if len(df) == 0:
        return 0

    return sum(slotfiller_match(row, slotfillers, strict)
        for (_, row), slotfillers in zip(df.iterrows(), pred_slot_fillers)) / len(df)

In [12]:
from sklearn.metrics import classification_report, precision_recall_fscore_support

result_rows = []

for ft in df_dev_results["run 1"]["Fallacy Types"].unique():
    def get_slot_filler_score(df, col, strict):
        return slot_filler_score(
             df[(df["Fallacy Types"] == ft) & (df["predicted_template_no"] == df["Template #"])],
             df[(df["Fallacy Types"] == ft) & (df["predicted_template_no"] == df["Template #"])]["predicted_slot_fillers"],
             strict=strict,
             )

    result_rows.append({
        "Fallacy Type": ft,
        "run 1 em": get_slot_filler_score(df_dev_results["run 1"], "predicted_template_no", strict=True),
        "run 2 em": get_slot_filler_score(df_dev_results["run 2"], "predicted_template_no", strict=True),
        "run 3 em": get_slot_filler_score(df_dev_results["run 3"], "predicted_template_no", strict=True),
        "run 4 em": get_slot_filler_score(df_dev_results["run 4"], "predicted_template_no", strict=True),
        "run 5 em": get_slot_filler_score(df_dev_results["run 5"], "predicted_template_no", strict=True),
        "run 6 em": get_slot_filler_score(df_dev_results["run 6"], "predicted_template_no", strict=True),
        "run 7 em": get_slot_filler_score(df_dev_results["run 7"], "predicted_template_no", strict=True),
        "run 8 em": get_slot_filler_score(df_dev_results["run 8"], "predicted_template_no", strict=True),
        "run 9 em": get_slot_filler_score(df_dev_results["run 9"], "predicted_template_no", strict=True),
        "run 10 em": get_slot_filler_score(df_dev_results["run 10"], "predicted_template_no", strict=True),
        "run 1 pm": get_slot_filler_score(df_dev_results["run 1"], "predicted_template_no", strict=False),
        "run 2 pm": get_slot_filler_score(df_dev_results["run 2"], "predicted_template_no", strict=False),
        "run 3 pm": get_slot_filler_score(df_dev_results["run 3"], "predicted_template_no", strict=False),
        "run 4 pm": get_slot_filler_score(df_dev_results["run 4"], "predicted_template_no", strict=False),
        "run 5 pm": get_slot_filler_score(df_dev_results["run 5"], "predicted_template_no", strict=False),
        "run 6 pm": get_slot_filler_score(df_dev_results["run 6"], "predicted_template_no", strict=False),
        "run 7 pm": get_slot_filler_score(df_dev_results["run 7"], "predicted_template_no", strict=False),
        "run 8 pm": get_slot_filler_score(df_dev_results["run 8"], "predicted_template_no", strict=False),
        "run 9 pm": get_slot_filler_score(df_dev_results["run 9"], "predicted_template_no", strict=False),
        "run 10 pm": get_slot_filler_score(df_dev_results["run 10"], "predicted_template_no", strict=False),
    })

result_rows.append({
    "Fallacy Type": "Overall",
    "run 1 em": np.mean([row["run 1 em"] for row in result_rows]),
    "run 2 em": np.mean([row["run 2 em"] for row in result_rows]),
    "run 3 em": np.mean([row["run 3 em"] for row in result_rows]),
    "run 4 em": np.mean([row["run 4 em"] for row in result_rows]),
    "run 5 em": np.mean([row["run 5 em"] for row in result_rows]),
    "run 6 em": np.mean([row["run 6 em"] for row in result_rows]),
    "run 7 em": np.mean([row["run 7 em"] for row in result_rows]),
    "run 8 em": np.mean([row["run 8 em"] for row in result_rows]),
    "run 9 em": np.mean([row["run 9 em"] for row in result_rows]),
    "run 10 em": np.mean([row["run 10 em"] for row in result_rows]),
    "run 1 pm": np.mean([row["run 1 pm"] for row in result_rows]),
    "run 2 pm": np.mean([row["run 2 pm"] for row in result_rows]),
    "run 3 pm": np.mean([row["run 3 pm"] for row in result_rows]),
    "run 4 pm": np.mean([row["run 4 pm"] for row in result_rows]),
    "run 5 pm": np.mean([row["run 5 pm"] for row in result_rows]),
    "run 6 pm": np.mean([row["run 6 pm"] for row in result_rows]),
    "run 7 pm": np.mean([row["run 7 pm"] for row in result_rows]),
    "run 8 pm": np.mean([row["run 8 pm"] for row in result_rows]),
    "run 9 pm": np.mean([row["run 9 pm"] for row in result_rows]),
    "run 10 pm": np.mean([row["run 10 pm"] for row in result_rows]),
})

pd.options.display.float_format = "{:.2f}".format
pd.DataFrame(result_rows).transpose()

,0,1,2,3,4
Fallacy Type,False Dilemma,Faulty Generalization,False Causality,Fallacy of Credibility,Overall
run 1 em,1.00,0.82,0.00,0.94,0.69
run 2 em,1.00,1.00,1.00,0.94,0.99
run 3 em,1.00,0.76,0.00,0.89,0.66
run 4 em,1.00,0.88,1.00,0.94,0.95
run 5 em,1.00,0.93,0.50,0.89,0.83
run 6 em,1.00,0.88,0.25,0.94,0.77
run 7 em,1.00,0.76,1.00,0.94,0.93
run 8 em,1.00,0.88,1.00,0.88,0.94
run 9 em,1.00,0.82,0.00,0.89,0.68


In [13]:
#Exact Match
FD = [result_rows[0]['run {} em'.format(i)] for i in range(1,11)]
false_dilemma = sum(FD) / len(FD)
fd_deviation = statistics.stdev(FD)

FG = [result_rows[1]['run {} em'.format(i)] for i in range(1,11)]
generalization = sum(FG) / len(FG)
fg_deviation = statistics.stdev(FG)

FC = [result_rows[2]['run {} em'.format(i)] for i in range(1,11)]
false_causality = sum(FC) / len(FC)
fc_deviation = statistics.stdev(FC)

FCred = [result_rows[3]['run {} em'.format(i)] for i in range(1,11)]
cred = sum(FCred) / len(FCred)
fcred_deviation = statistics.stdev(FCred)

overall = [result_rows[4]['run {} em'.format(i)] for i in range(1,11)]
average = sum(overall) / len(overall)
ovl_deviation = statistics.stdev(overall)

print(false_dilemma)
print(generalization)
print(false_causality)
print(cred)
print(average)

1.0
0.8670728291316527
0.5083333333333333
0.911231510147919
0.8216594181532262


In [14]:
print(fd_deviation)
print(fg_deviation)
print(fc_deviation)
print(fcred_deviation)
print(ovl_deviation)

0.0
0.07524136626942454
0.4521041341860835
0.03743827497046058
0.12314277918432681


In [15]:
#Partial Match
FD = [result_rows[0]['run {} pm'.format(i)] for i in range(1,11)]
false_dilemma = sum(FD) / len(FD)
fd_deviation = statistics.stdev(FD)

FG = [result_rows[1]['run {} pm'.format(i)] for i in range(1,11)]
generalization = sum(FG) / len(FG)
fg_deviation = statistics.stdev(FG)

FC = [result_rows[2]['run {} pm'.format(i)] for i in range(1,11)]
false_causality = sum(FC) / len(FC)
fc_deviation = statistics.stdev(FC)

FCred = [result_rows[3]['run {} pm'.format(i)] for i in range(1,11)]
cred = sum(FCred) / len(FCred)
fcred_deviation = statistics.stdev(FCred)

overall = [result_rows[4]['run {} pm'.format(i)] for i in range(1,11)]
average = sum(overall) / len(overall)
ovl_deviation = statistics.stdev(overall)

print(false_dilemma)
print(generalization)
print(false_causality)
print(cred)
print(average)

1.0
0.9286764705882353
0.7666666666666667
0.911231510147919
0.9016436618507052


In [16]:
print(fd_deviation)
print(fg_deviation)
print(fc_deviation)
print(fcred_deviation)
print(ovl_deviation)

0.0
0.07797431161490644
0.34426518632954817
0.03743827497046058
0.08861768198992674
